In [11]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion

TRAIN_DIR = 'train_set'
TEST_DIR = 'test_set'
IMG_SIZE = 28
LR = 1e-3

In [12]:
def label_img(img):
    image_label = img.split('.')
    name = image_label[0]
    # checking if the signature is forged or genuine
    #                            
    
    if name[4:7] == name[-3:]:
        return 1
  
    elif name[4:7] != name[-3:]:
        return 0

In [13]:
def person_img(img):
    label = img.split('.')
    k = label[0]
    return [k[-3:]]

In [14]:
def create_features(DIR):
    features = []
    labels = []
    for img in tqdm(os.listdir(DIR)):
        label = label_img(img)
        person = person_img(img)
       
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        features.append([img])
        labels.append(np.array(label))
    #shuffle(training_data)
    #np.save('train_data.npy', training_data)
    return features

In [15]:
train_features  = np.array(create_features(TRAIN_DIR))
test_features  = np.array(create_features(TEST_DIR))
print test_features.shape
train_features_reshaped = train_features.reshape(-1, 28,28, 1)
test_features_reshaped = test_features.reshape(-1, 28,28, 1)
print test_features_reshaped.shape
print train_features_reshaped.shape

train_features_reshaped = train_features_reshaped.astype('float32')
test_features_reshaped = test_features_reshaped.astype('float32')
train_features_reshaped = train_features_reshaped / 255.
test_features_reshaped = test_features_reshaped / 255.

100%|██████████| 121/121 [00:01<00:00, 61.52it/s]

(121, 1, 28, 28)
(121, 28, 28, 1)
(179, 28, 28, 1)


In [16]:
def create_labels(DIR):
    labels = []
    for img in tqdm(os.listdir(DIR)):
        label = label_img(img)
        labels.append(np.array(label))
    return labels

In [17]:
train_labels = np.array(create_labels(TRAIN_DIR))
test_labels = np.array(create_labels(TEST_DIR))
print test_labels.shape

100%|██████████| 121/121 [00:00<00:00, 32265.93it/s]

(121,)


In [18]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
train_labels_one_hot = to_categorical(train_labels)

In [26]:
test_labels_one_hot = to_categorical(test_labels)
print test_labels_one_hot.shape

(121, 2)


In [27]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

batch_size = 64
epochs = 20
num_classes = 2


In [28]:
fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(28,28,1),padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(Dense(num_classes, activation='softmax'))


In [29]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
fashion_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 128)         73856     
__________

In [30]:
fashion_train = fashion_model.fit(train_features_reshaped, train_labels_one_hot, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(test_features_reshaped, test_labels_one_hot))

Train on 179 samples, validate on 121 samples
Epoch 1/20
179/179 [==============================] - 2s 14ms/step - loss: 0.8609 - acc: 0.4358 - val_loss: 0.6939 - val_acc: 0.5041
Epoch 2/20
179/179 [==============================] - 0s 3ms/step - loss: 0.7005 - acc: 0.5028 - val_loss: 0.7011 - val_acc: 0.4959
Epoch 3/20
179/179 [==============================] - 0s 3ms/step - loss: 0.6948 - acc: 0.5028 - val_loss: 0.6933 - val_acc: 0.4793
Epoch 4/20
179/179 [==============================] - 1s 3ms/step - loss: 0.6948 - acc: 0.5028 - val_loss: 0.6942 - val_acc: 0.5041
Epoch 5/20
179/179 [==============================] - 1s 4ms/step - loss: 0.6959 - acc: 0.4972 - val_loss: 0.6929 - val_acc: 0.5207
Epoch 6/20
179/179 [==============================] - 1s 4ms/step - loss: 0.6932 - acc: 0.4804 - val_loss: 0.6930 - val_acc: 0.4959
Epoch 7/20
179/179 [==============================] - 1s 3ms/step - loss: 0.6926 - acc: 0.5363 - val_loss: 0.6928 - val_acc: 0.5289
Epoch 8/20
179/179 [=========